In [11]:
import csv
import re
import pandas as pd
from pathlib import Path
import filecmp
from bitstruct import unpack_from as upf

import PC_Fns

In [16]:
nsvf_file = Path(r"C:\Users\ucasbwh\OneDrive - University College London\PanCam Documents\SWIS\200131 SWIS L2 3.1.12\NSVF Logs\PanCam_v2.0B_Delay10_RUN_1\Test.log")
nsvf_file = Path(r"C:\Users\ucasbwh\OneDrive - University College London\PanCam Documents\SWIS\200131 SWIS L2 3.1.12\NSVF Logs\PanCam_v2.0B_Delay10_RUN_1\Router_A_packet.log")

if not nsvf_file.exists():
    print("Yeah not a file")
else:
    print("Yeah is a file")

test_file = Path(r"C:\Users\ucasbwh\OneDrive - University College London\PanCam Documents\SWIS\200131 SWIS L2 3.1.12\NSVF Logs\PanCam_v2.0B_Delay10_RUN_1\Output.log")
hs_file = Path(r"C:\Users\ucasbwh\OneDrive - University College London\PanCam Documents\SWIS\200131 SWIS L2 3.1.12\NSVF Logs\PanCam_v2.0B_Delay10_RUN_1\H&S.log")
hk_file = Path(r"C:\Users\ucasbwh\OneDrive - University College London\PanCam Documents\SWIS\200131 SWIS L2 3.1.12\NSVF Logs\PanCam_v2.0B_Delay10_RUN_1\HK.log")
tc_file = Path(r"C:\Users\ucasbwh\OneDrive - University College London\PanCam Documents\SWIS\200131 SWIS L2 3.1.12\NSVF Logs\PanCam_v2.0B_Delay10_RUN_1\TC.log")

Yeah is a file


In [110]:
# Check initial format is as expected, #[IN=..], #[SZ=..], [EOP] at end

# Checks to perform
# Is a file and readable
# That each line:
#       ends with "[EOP]"
#       contains [IN=$I$] with $I$ some integer
#       contains [SZ=$I$] with $I$ some integer
#       that the size in [SZ..] matches packet size

in_re = re.compile("^\[IN=[0-9]+\]$")
sz_re = re.compile("^\[SZ=[0-9]+\]$")

# First check that #[IN=..]
outputfile = open(test_file, 'w')
linewriter = csv.writer(outputfile, delimiter=' ', lineterminator='\r')
hsfile = open(hs_file, 'w')
hs_writer = csv.writer(hsfile, delimiter=' ', lineterminator='\r')
hkfile = open(hk_file, 'w')
hk_writer = csv.writer(hkfile, delimiter=' ', lineterminator='\r')
tcfile = open(tc_file, 'w')
tc_writer = csv.writer(tcfile, delimiter=' ', lineterminator='\r')

with open(nsvf_file) as logfile:
    reader = csv.reader(logfile, delimiter=' ')
    for row in reader:
        #print(row)
        if row[-1] != '[EOP]': print("Error Found")
        if not in_re.match(row[1]): print("No Match for [IN]")
        if not sz_re.match(row[2]): print("No Match for [SZ]")
        row_size = int(row[2][4:-1])
        if row_size != len(row[4:-1]): print("Packet does not match expected size")

        # Filter by packet types
        # If SZ = 45 and start = [80 00 00 00] then H&S
        # Then determine TM Header

        log_addr = int(row[8], 16)
        if log_addr == 0x41:
            if row_size == 45:
                row_reduced = row
                row_reduced[1:4] = []
                row_reduced[-1:] = []
                row_reduced[0] += ';'
                hs_writer.writerow(row)
            elif row_size == 8:
                tc_writer.writerow(row)
            else:
                tm_type = int(row[17], 16) >> 2
                if tm_type < 2:
                    hk_writer.writerow(row)
                else:
                    linewriter.writerow(row)

outputfile.close()
hsfile.close()
hkfile.close()
tcfile.close()

print("Done")


Done


In [111]:
pc_hk_addr = 0x80000000
pc_hk_lens = [0, 72, 88]
pc_sci_addr = 0xC0000000
pc_sci_len  = [0x200030]

In [113]:
# HS Verify
# Check that the HK address is always 0x80 00 00 00
# Check that the HK counter is always increasing
# Check that the Sci address is always 0xC0 00 00 00
# Check that the Sci counter increases
# Count number of images generated
# Ensure LDT count is always <7

hs_head = ['Time', 'RAW']
hs = pd.read_csv(hs_file, sep=';', header=None, names=hs_head)

#DL['RAW'] = DL.RAW_DATA.apply(lambda x: x[38:-4])

raw = hs['RAW'].apply(lambda x: bytearray.fromhex(x))

# HS Decode
hs['HK_Addr'] = PandUPF(raw, 'u32', 12, 0)
hs['HK_Len'] = PandUPF(raw, 'u16', 16, 0)
hs['HK_Cnt'] = PandUPF(raw, 'u16', 18, 0)
hs['Sci_Addr'] = PandUPF(raw, 'u32', 20, 0)
hs['Sci_Len'] = PandUPF(raw, 'u24', 24, 0)
hs['LDT'] = PandUPF(raw, 'u8', 27, 0)
hs['Sci_Cnt'] = PandUPF(raw, 'u16', 28, 0)

# Verify HK Address is always 0x80 00 00 00
# Check first value is as expected
verify = pd.DataFrame()
err_df = pd.DataFrame()
verify['HK_Addr'] = ~(hs['HK_Addr'] == pc_hk_addr)
err_df = hs[verify['HK_Addr']] 
if err_df.shape[0] != 0:
    print("HS HK Address not as expected %d occurances", err_df.shape[0])
    print(err_df)

# Verify the HK Length is either 72 or 88 bytes
verify['HK_Len'] = ~hs['HK_Len'].isin(pc_hk_lens)
err_df = hs[verify['HK_Len']]
if err_df.shape[0] != 0:
    print("HS HK unexpected length %d occurances", err_df.shape[0])
    print(err_df)

# Verify HK counter is always increasing
verify['HK_Cnt'] = hs['HK_Cnt'].diff() < 0
err_df = hs[verify['HK_Cnt']]
if err_df.shape[0] != 0:
    print("HS HK Count not increasing, %d occurances", err_df.shape[0])
    for row in err_df.index.values:
        print("Occurance")
        print(hs.iloc[row-1:row+2])

# Verify Sci Address is always 0xC0 00 00 00
# Check first value is as expected
verify['Sci_Addr'] = ~(hs['Sci_Addr'] == pc_sci_addr)
err_df = hs[verify['Sci_Addr']]
if err_df.shape[0] != 0:
    print("HS Sci Address not as expected %d occurances", err_df.shape[0])
    print(err_df)

# Verify the Sci Length is always 2,097,200 bytes
verify['Sci_Len'] = ~hs['Sci_Len'].isin(pc_sci_len)
err_df = hs[verify['Sci_Len']]
if err_df.shape[0] != 0:
    print("HS Sci unexpected length %d occurances", err_df.shape[0])
    print(err_df) 

# Verify the LDT Ctrl is always <7
verify['LDT'] = ~(hs['LDT'] < 8)
err_df = hs[verify['LDT']]
if err_df.shape[0] != 0:
    print("HS LDT not in expected range of 0 to 7, %d occurances", err_df.shape[0])
    print(err_df)

# Check that the Sci counter is always increasing
verify['Sci_Cnt'] = hs['Sci_Cnt'].diff() < 0
err_df = hs[verify['Sci_Cnt']]
if err_df.shape[0] != 0:
    print("HS Sci count not increasing, %d occurances", err_df.shape[0])
    for row in err_df.index.values:
        print("Occurance")
        print(hs.iloc[row-1:row+2])

# Count number of images generated
# First find last count
img_cnt = hs['Sci_Cnt'].iloc[-1]

# Find all instances where it has reset and add to total
cnt_rst = hs.Sci_Cnt[hs['Sci_Cnt'].diff(-1) > 0]
img_cnt += cnt_rst.sum()
print("A total of %d images generated over session", img_cnt)

HS HK Count not increasing, %d occurances 1
Occurance
                   Time                                                RAW  \
1104  [515:838:009:386]   21 01 0C 00 41 00 B9 00 00 00 20 D5 80 00 00 ...   
1105  [533:620:009:386]   21 01 0C 00 41 00 31 00 00 00 20 DF 80 00 00 ...   
1106  [533:820:009:386]   21 01 0C 00 41 00 32 00 00 00 20 25 80 00 00 ...   

         HK_Addr  HK_Len  HK_Cnt    Sci_Addr  Sci_Len  LDT  Sci_Cnt  
1104  2147483648      88     219  3221225472  2097200    0        1  
1105  2147483648       0       0  3221225472  2097200    0        0  
1106  2147483648       0       0  3221225472  2097200    0        0  
HS Sci count not increasing, %d occurances 1
Occurance
                   Time                                                RAW  \
1104  [515:838:009:386]   21 01 0C 00 41 00 B9 00 00 00 20 D5 80 00 00 ...   
1105  [533:620:009:386]   21 01 0C 00 41 00 31 00 00 00 20 DF 80 00 00 ...   
1106  [533:820:009:386]   21 01 0C 00 41 00 32 00 00 00 20 25 80

In [106]:
cnt_rst = hs.Sci_Cnt[hs['Sci_Cnt'].diff(-1) >0]
print(cnt_rst.sum())

3


In [44]:
proc_dir = Path(r"C:\Users\ucasbwh\Desktop\NSVF Dev\PROC")

sci_file = PC_Fns.Find_Files(proc_dir, 'Sci.txt')[0]
num_lines = sum(1 for line in open(sci_file))
print("Number of lines = ", num_lines)

write_file = sci_file.parent / "RAW.bin"
f = open(write_file, 'w+b')

with open(sci_file) as txtfile:
    reader = csv.reader(txtfile, delimiter=' ')
    for row in reader:
        test = row[16:-2]
        output = ''.join(test)
        binary_format = bytearray.fromhex(output)
        f.write(binary_format)

f.close()

Number of lines =  14


In [57]:
num_imgs = 2
img_name = list(range(num_imgs))
print(img_name)
print(str(img_name[0]).zfill(2))

[0, 1]
00


In [40]:
## Compare science files
proc_dir = Path(r"C:\Users\ucasbwh\Desktop\NSVF Dev\PROC")
sci_files = PC_Fns.Find_Files(proc_dir, '*.pci_raw')
print(len(sci_files))

ref_dir = Path().parent.absolute().parent / "data" / "SWIS_Reference"

for sci in sci_files:
    #Chop off header
    gen = open(sci, 'rb')
    header = gen.read(48)
    cam =  upf('u2', header, offset=130)[0]   

    # Determine Cam Type
    if cam == 1:
        # WACL Image Reference
        ref = ref_dir / "pfm_wacl.raw"
    else:
        print("Wrong Type")
    
    #Generate temporary file
    tmp_file = sci.with_suffix(".tmp")
    tmp = open(tmp_file, 'wb')
    tmp.write(gen.read())
    tmp.close()

    #Compare files
    if filecmp.cmp(tmp_file, ref, shallow=False):
        print("Files Match")
    else:
        print("Files do not match")

    tmp_file.unlink()
    gen.close()

2
Files do not match
Files Match


In [24]:
p = Path().parent.absolute().parent / "data" / "SWIS_Reference"
print(p)
print(p.is_dir())

c:\Users\ucasbwh\Documents\GitHub\PanCam-Data-Processing-Tools\data\SWIS_Reference
True
